In [1]:
from tkinter import *
from owlready2 import *
import pandas as pd
import datetime, math
import rdflib

In [2]:
onto = get_ontology('main-ResearchMentorOntology.owl')
vivoNS = onto.get_namespace("http://vivoweb.org/ontology/core")
meshNS = onto.get_namespace("http://phenomebrowser.net/ontologies/mesh/mesh.owl#")
onto.load()

[core.StanfordGeneticsandBiologyDepartment, core.StanfordGeneticsandBiomedicalInformaticsTrainingProgDepartment, core.StanfordGeneticsandMicrobiologyDepartment, core.StanfordGeneticsandPediatricsDepartment, core.StanfordGeneticsandPrograminBiomedicalInformaticsDepartment, core.StanfordGeologicalSciencesDepartment, core.StanfordGeologicalSciencesandCenterforInternationalSecurityandCooperatiDepartment, core.StanfordGeologicalandEnvironmentalSciencDepartment, core.StanfordGeologicalandEnvironmentalSciencesDepartment, core.StanfordGeologyDepartment, core.StanfordInternalMediciDepartment, core.StanfordInterventionalRadiologyDepartment, core.StanfordLinguisticsDepartment, core.StanfordManagementScienceandEngineeringDepartment, core.StanfordManagementScienceandEngineeringatStanfordUniversityDepartment, core.StanfordMaterialScienceandEngineeringDepartment, core.StanfordHealthMediciDepartment, core.StanfordHealthPolicyandResearchDepartment, core.StanfordHealthResearchDepartment, core.StanfordHe

In [ ]:
class ResearchMentorApp:
    def __init__(self, master):
        # preload ontology individuals
        self.onto, self.vivoNS = self.load_ontology()
        self.load_individuals()
        print(self.doctoral_students)
        
        frame = Frame(master)
        frame.pack()
        
        # Quit and enter buttons
        self.button = Button(frame, text="QUIT", fg="red", command=frame.quit)
        self.button.pack(side=RIGHT)
        
        self.button = Button(frame, text="ENTER", fg="red", command=frame.quit)
        self.button.pack(side=RIGHT)
        
        # search boxes
        SEARCH_BY_OPTIONS = ["Affliation","MESH"]
        self.search_by_var = StringVar(master)
        self.search_by_var.set(SEARCH_BY_OPTIONS[0])
        self.search_by_option = OptionMenu(master, self.search_by_var,*SEARCH_BY_OPTIONS)
        self.search_by_option.pack()
        
        depart_indivs = self.get_departments(onto,vivoNS)
        DEPART_OPTIONS = [indiv.name for indiv in depart_indivs]
        self.depart_var = StringVar(master)
        self.depart_var.set(DEPART_OPTIONS[0])
        self.depart_option = OptionMenu(master, self.depart_var, *DEPART_OPTIONS)
        self.depart_option.pack()
    
    def load_individuals(self):
        self.faculty_members = self.vivoNS.FacultyMember.instances()
        self.undergraduate_students = self.vivoNS.UndergraduateStudent.instances()
        self.doctoral_students = self.onto.DoctoralStudent.instances()
#         self.organizations.extend(onto.search(type=))
        self.documents = self.vivoNS.AcademicArticle.instances()
        self.mesh_terms = self.onto.Mesh.instances()
        
        
    def load_graph():
        g = rdflib.Graph()
        g.parse("main-ResearchMentorOntology.owl")
        return g

    def load_ontology(self):
        onto = get_ontology('main-ResearchMentorOntology.owl')
        vivoNS = onto.get_namespace("http://vivoweb.org/ontology/core")
        meshNS = onto.get_namespace("http://phenomebrowser.net/ontologies/mesh/mesh.owl#")
        onto.load()
        return onto, vivoNS
    
    def get_departments(self, onto, vivoNS):
        res = onto.search(type=vivoNS.Division)
        return res

root = Tk()
app = ResearchMentorApp(root)
root.mainloop()
root.destroy()

In [5]:
class App:
    def __init__(self, master):
        frame = Frame(master)
        frame.pack()
        
        self.button = Button(frame, text="QUIT", fg="red", command=frame.quit)
        self.button.pack(side=LEFT)
        self.hi_there = Button(frame, text="Hellow", command=self.say_hi)
        self.hi_there.pack(side=LEFT)
    def say_hi(self):
        print("hi there, everyone!")

root = Tk()
app = App(root)
root.mainloop()
root.destroy()

hi there, everyone!


In [3]:
# hello world app
# root = Tk()
# w = Label(root, text="Hello, world!")
# w.pack()
# root.mainloop()